In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import re
import json
import pdb
from bs4 import BeautifulSoup
import personal_keys

# fav_funcs.py

## fav_request
makes request to API, returns json file of user

In [ ]:
def fav_request(username, page_size, page):
    favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format(username)
    favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':page_size, 'page':page})
    return favs.json()

### example output:

In [9]:
fav_request("katec125", 2, 0).keys()

dict_keys(['favorites', 'paginator'])

## create_fav_list
input fav json, output list of fav pattern codes

In [10]:
def create_fav_list(fav_request):
    fav_list = []
    favorites = fav_request['favorites']
    for item in range(0,len(favorites)):
        if favorites[item]['favorited'] is not None:
            if 'pattern_id' in favorites[item]['favorited'].keys():
                fav_list.append(favorites[item]['favorited']['pattern_id'])
            elif 'id' in favorites[item]['favorited'].keys():
                fav_list.append(favorites[item]['favorited']['id'])
    return [code for code in fav_list if code is not None]

### example output: 

In [11]:
fav_json = fav_request("katec125", 2, 0)
create_fav_list(fav_json)

[1017571, 777372]

## get_favs_list
* Input is username, output is list of pattern ids in a person's favorites
* Essentially combination of fav request and create_fav_list
* If the user has over 500 favorites, then only returns the first 500. This is done so other functions don't take forever

In [12]:
def get_favs_list(username):
    favs = fav_request(username, 100, 1)
    fav_list = create_fav_list(favs)
    if favs['paginator']['page_count']>1:
        page_number = 2
        if favs['paginator']['last_page'] > 5:
            last_page = 5
        else:
            last_page = favs['paginator']['last_page']
        while page_number <= last_page:
            new_request_favs = fav_request(username,100, page_number)
            fav_list.extend(create_fav_list(new_request_favs))
            page_number += 1
    return fav_list

### Example output 

In [14]:
get_favs_list("katec125")[:10]

[1017571,
 777372,
 1002216,
 998422,
 877616,
 990541,
 966823,
 444597,
 888409,
 420814]

## fav_dict
* Returns a dict that is just pattern code and 1 for every value
* This is done so we can then do the proj rating func to create user data column

In [15]:
def fav_dict (username):
    fav_list = get_favs_list(username)
    return {code:1 for code in fav_list}

In [21]:
fav_dict("katec125")[1017571]

1

# proj_funcs.py

## proj_json
returns json file with all the info about user patterns

In [22]:
def proj_json(username):
    projects_url = 'https://api.ravelry.com/projects/{}/list.json'.format(username)
    projects = requests.get(projects_url, 
                        auth = (personal_keys.username(),personal_keys.password()))
    return projects.json()

### Example output:

In [23]:
proj_json("katec125").keys()

dict_keys(['projects', 'collections', 'paginator'])

## project_list
Input is a json obtained in the proj_json function, output is a list of pattern ids for user's projects

In [24]:
def project_list(p_json):
    proj_list = []
    proj_list.extend([p_json['projects'][item]['pattern_id'] 
                    for item in range(0,len(p_json['projects']))])
    return proj_list

### Example output:

In [25]:
p_json = proj_json("katec125")
project_list(p_json)[:10]

[669230, 118820, 490330, 103767, 91776, 216824, 990541, 582245, 921518, 497647]

## get_project_list_from_username

combination of the two previous functions


In [26]:
def get_project_list_from_username(username):
    p_json = proj_json(username)
    return project_list(p_json)

### Example output: 

In [27]:
get_project_list_from_username("katec125")[:10]

[669230, 118820, 490330, 103767, 91776, 216824, 990541, 582245, 921518, 497647]

## project_rating
* returns dictionary of project codes and user ratings, 
* default rating is 3 if none is given, 
* otherwise it's on a scale from 1-5 (hence the +1)

In [28]:
def project_rating(username):
    code_list = get_project_list_from_username(username)
    rating_list = [3 if proj_json(username)['projects'][i]['rating'] == None 
                    else proj_json(username)['projects'][i]['rating']+1 
                    for i in range(0,len(proj_json("katec125")['projects']))]
    return dict(zip(code_list, rating_list))

Example output:

In [29]:
project_rating("katec125")

{669230: 3,
 118820: 3,
 490330: 3,
 103767: 4,
 91776: 3,
 216824: 3,
 990541: 3,
 582245: 3,
 921518: 3,
 497647: 3,
 828182: 3,
 833778: 3,
 807214: 3,
 497856: 3,
 617705: 3,
 722427: 3,
 714457: 3,
 676427: 3,
 226669: 3}

## user_data 
* creates dictionary where the keys are all user's favorites and projects
* all favorites get a 1 assigned
* projects get either their rating or 3 assigned
* the final dict is turned into a string so it can combine with the existing user profile df

In [30]:
def user_data(username):
    user_data = fav_dict(username)
    user_data.update(project_rating(username))
    str_user_data_dict = {str(k):v for k,v in user_data.items()}
    return str_user_data_dict

Example output:

In [31]:
user_data("katec125")

{'1017571': 1,
 '777372': 1,
 '1002216': 1,
 '998422': 1,
 '877616': 1,
 '990541': 3,
 '966823': 1,
 '444597': 1,
 '888409': 1,
 '420814': 1,
 '790875': 1,
 '409979': 1,
 '538253': 1,
 '387775': 1,
 '994876': 1,
 '993218': 1,
 '975032': 1,
 '995685': 1,
 '731341': 1,
 '984587': 1,
 '384733': 1,
 '866947': 1,
 '932512': 1,
 '893880': 1,
 '986145': 1,
 '807636': 1,
 '936349': 1,
 '918043': 1,
 '907192': 1,
 '843846': 1,
 '866773': 1,
 '686988': 1,
 '966033': 1,
 '968150': 1,
 '869199': 1,
 '351173': 1,
 '824391': 1,
 '921518': 3,
 '843810': 1,
 '855203': 1,
 '925240': 1,
 '929139': 1,
 '570677': 1,
 '483117': 1,
 '639396': 1,
 '754362': 1,
 '824302': 1,
 '886241': 1,
 '561403': 1,
 '171113': 1,
 '57264': 1,
 '772466': 1,
 '497126': 1,
 '680396': 1,
 '868477': 1,
 '868478': 1,
 '497647': 3,
 '582257': 1,
 '377778': 1,
 '593997': 1,
 '838914': 1,
 '742300': 1,
 '858134': 1,
 '855568': 1,
 '763885': 1,
 '830861': 1,
 '853849': 1,
 '690209': 1,
 '843963': 1,
 '804284': 1,
 '853145': 1,
 '743

# friend_funcs.py

### Example output:

### Example output:

### Example output:

# search_functions.py 

### Example output:

### Example output:

### Example output:

# yarn_weights.py

### Example output:

# pattern_attr_funcs.py

### Example output:

### Example output:

### Example output:

### Example output:

### Example output:

### Example output:

# count_df_funcs.py


### Example output:

### Example output:

### Example output:

### Example output:

### Example output: